In [5]:
!pip install gradio==3.44 googletrans==4.0.0-rc1 pytesseract Pillow PyMuPDF

In [6]:
import gradio as gr
from googletrans import Translator
import pytesseract
from PIL import Image
import fitz  # PyMuPDF
import tempfile

In [7]:
language_map = {
    "Afrikaans": "af", "Albanian": "sq", "Amharic": "am", "Arabic": "ar", "Armenian": "hy",
    "Assamese": "as", "Azerbaijani": "az", "Basque": "eu", "Belarusian": "be", "Bengali": "bn",
    "Bosnian": "bs", "Bulgarian": "bg", "Burmese": "my", "Catalan": "ca", "Cebuano": "ceb",
    "Chichewa": "ny", "Chinese (Simplified)": "zh-cn", "Chinese (Traditional)": "zh-tw", "Corsican": "co",
    "Croatian": "hr", "Czech": "cs", "Danish": "da", "Dutch": "nl", "English": "en",
    "Esperanto": "eo", "Estonian": "et", "Filipino": "tl", "Finnish": "fi", "French": "fr",
    "Galician": "gl", "Georgian": "ka", "German": "de", "Greek": "el", "Gujarati": "gu",
    "Haitian Creole": "ht", "Hausa": "ha", "Hawaiian": "haw", "Hebrew": "he", "Hindi": "hi",
    "Hmong": "hmn", "Hungarian": "hu", "Icelandic": "is", "Igbo": "ig", "Indonesian": "id",
    "Irish": "ga", "Italian": "it", "Japanese": "ja", "Javanese": "jv", "Kannada": "kn",
    "Kazakh": "kk", "Khmer": "km", "Kinyarwanda": "rw", "Korean": "ko", "Kurdish (Kurmanji)": "ku",
    "Kyrgyz": "ky", "Lao": "lo", "Latin": "la", "Latvian": "lv", "Lithuanian": "lt",
    "Luxembourgish": "lb", "Macedonian": "mk", "Malagasy": "mg", "Malay": "ms", "Malayalam": "ml",
    "Maltese": "mt", "Maori": "mi", "Marathi": "mr", "Mongolian": "mn", "Nepali": "ne",
    "Norwegian": "no", "Pashto": "ps", "Persian": "fa", "Polish": "pl", "Portuguese": "pt",
    "Punjabi": "pa", "Romanian": "ro", "Russian": "ru", "Samoan": "sm", "Scots Gaelic": "gd",
    "Serbian": "sr", "Sesotho": "st", "Shona": "sn", "Sindhi": "sd", "Sinhala": "si",
    "Slovak": "sk", "Slovenian": "sl", "Somali": "so", "Spanish": "es", "Sundanese": "su",
    "Swahili": "sw", "Swedish": "sv", "Tagalog": "tl", "Tamil": "ta", "Telugu": "te",
    "Thai": "th", "Turkish": "tr", "Ukrainian": "uk", "Urdu": "ur", "Uzbek": "uz",
    "Vietnamese": "vi", "Welsh": "cy", "Xhosa": "xh", "Yiddish": "yi", "Yoruba": "yo",
    "Zulu": "zu"
}

# Histories
text_history = []
doc_history = []
image_history = []

In [8]:
# Text translation
def translate_text(text, target_lang):
    code = language_map.get(target_lang, "en")
    translator = Translator()
    translated = translator.translate(text, dest=code)
    return translated.text

# OCR for images
def ocr_image(file):
    img = Image.open(file)
    return pytesseract.image_to_string(img)

# Document translation
def translate_document(file, target_lang):
    text = ""
    if file.name.endswith(".pdf"):
        doc = fitz.open(file.name)
        for page in doc:
            text += page.get_text()
    else:
        text = file.read().decode("utf-8")
    return translate_text(text, target_lang)

In [9]:
text_history = []
doc_history = []
image_history = []

In [10]:
with gr.Blocks() as demo:

    gr.Markdown("<h1 style='text-align:center;color:#4B4BFF;'>AI Multi-Language Translator</h1>")
    gr.Markdown("<p style='text-align:center;font-size:16px;'>Translate Text, Documents, and Images instantly with AI.</p>")

    with gr.Tabs():

        # Text Tab
        with gr.TabItem("Text"):
            txt_input = gr.Textbox(label="Enter text")
            lang_dropdown = gr.Dropdown(list(language_map.keys()), label="Target Language")
            translate_btn = gr.Button("Translate")
            txt_output = gr.Textbox(label="Translated Text")
            text_hist_box = gr.Textbox(label="Translation History", interactive=False)

            def translate_text_tab(text, lang):
                if not text.strip(): return "", ""
                translated = translate_text(text, lang)
                text_history.insert(0, f"{text} → {translated} ({lang})")
                if len(text_history) > 10: text_history.pop()
                return translated, "\n".join(text_history)

            translate_btn.click(translate_text_tab, [txt_input, lang_dropdown], [txt_output, text_hist_box])

        # Document Tab
        with gr.TabItem("Document"):
            doc_input = gr.File(label="Upload PDF/TXT")
            doc_lang_dropdown = gr.Dropdown(list(language_map.keys()), label="Target Language")
            doc_translate_btn = gr.Button("Translate Document")
            doc_output = gr.Textbox(label="Translated Text")
            doc_hist_box = gr.Textbox(label="Document History", interactive=False)

            def translate_doc_tab(file, lang):
                if not file: return "", ""
                translated = translate_document(file, lang)
                doc_history.insert(0, f"{file.name} → {translated[:50]}... ({lang})")
                if len(doc_history) > 10: doc_history.pop()
                return translated, "\n".join(doc_history)

            doc_translate_btn.click(translate_doc_tab, [doc_input, doc_lang_dropdown], [doc_output, doc_hist_box])

        # Image OCR Tab
        with gr.TabItem("Image OCR"):
            img_input = gr.File(label="Upload Image")
            img_lang_dropdown = gr.Dropdown(list(language_map.keys()), label="Target Language")
            img_translate_btn = gr.Button("Translate Image Text")
            img_output = gr.Textbox(label="Translated Text")
            img_hist_box = gr.Textbox(label="Image OCR History", interactive=False)

            def translate_img_tab(file, lang):
                if not file: return "", ""
                extracted_text = ocr_image(file)
                translated = translate_text(extracted_text, lang)
                image_history.insert(0, f"{file.name} → {translated[:50]}... ({lang})")
                if len(image_history) > 10: image_history.pop()
                return translated, "\n".join(image_history)

            img_translate_btn.click(translate_img_tab, [img_input, img_lang_dropdown], [img_output, img_hist_box])

demo.launch(share=True)

IMPORTANT: You are using gradio version 3.44.0, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://85719f14e38c76edf0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
